![head.png](https://github.com/iwh-halle/FinancialDataAnalytics/blob/master/figures/head.jpg?raw=1)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

# Homework

You will need a Git/GitHub repository to submit your course deliverables. Consult [**slides.ipynb**](https://github.com/iwh-halle/FinancialDataAnalytics) for help with the tasks below! If you need further assistance, do not hesitate to open a Q&A at https://github.com/cafawo/FinancialDataAnalytics/discussions

### Task: 

The liquidity position of a firm measured in million is a **generalized Wiener process** with a drift of $0.1$ per month and a variance of $\sigma^2 = 0.16$ per month. The initial cash position is $2.0$. Calculate:

1. 	the expected value and standard deviation in one, six and 12 months.
2.	What is the probability of a negative cash position in 6 and 12 months? 

In [7]:
from scipy.stats import norm

# Parameters
drift_per_month = 0.1  
variance_per_month = 0.16 
initial_cash_position = 2  

# Calculate standard deviation from the variance
standard_deviation_per_month = variance_per_month**0.5

# Time intervals in months for the calculations
time_intervals = [1, 6, 12]

results = []
for months in time_intervals:
    # Expected value of cash position at time point
    expected_cash_position = drift_per_month * months + initial_cash_position
    # Standard deviation of cash position at time point
    cash_position_std_dev = standard_deviation_per_month * months**0.5
    # Probability that cash position is negative at time point
    probability_negative_cash = norm.cdf((0 - expected_cash_position) / cash_position_std_dev)
    
    # Storing results
    results.append({
        "Expected Cash Position": expected_cash_position,
        "Standard Deviation": cash_position_std_dev,
        "Probability of Negative Cash": probability_negative_cash
    })


#1 
print_counter = 0
for result in results:
    print(f"Month {time_intervals[print_counter]}:\n\nexpected value: {result['Expected Cash Position']:.2f}\nstandard deviation: {result['Standard Deviation']:.2f}\n")
    print_counter += 1

#2
print_counter = 0
for result in results:
    if print_counter == 0:          #skip month 1
        print_counter += 1
        continue
    print(f"The result for month {time_intervals[print_counter]} is a probability of negative cash of {result['Probability of Negative Cash'] * 100:.2f}%")
    print_counter += 1

Month 1:

expected value: 2.10
standard deviation: 0.40

Month 6:

expected value: 2.60
standard deviation: 0.98

Month 12:

expected value: 3.20
standard deviation: 1.39

The result for month 6 is a probability of negative cash of 0.40%
The result for month 12 is a probability of negative cash of 1.05%


### Task: 

The cash flow of a [call option](https://en.wikipedia.org/wiki/Call_option) with strike $K$ at maturity $T$ is given by

$$
max(S_T - K, 0) = (S_T - K)^+
$$

where $S_T$ is the price of the underlying at $T$. The price of the option under the [risk-neutral measure](https://en.wikipedia.org/wiki/Risk-neutral_measure) $\mathbb{Q}$ is simply its discounted expected value
$$
\mathbb{E}^\mathbb{Q}[(S_T - K)^+] e^{-rT}.
$$


Calculate the price of the option, using:
1. numerical integration and
2. Monte carlo simulation.

For you calculations, assume that todays price of the underlying is $S_0 = 220$, the strike is $K = 220$, volatility is $\sigma = 0.98$, the risk free rate is $r = 10\%$ (continuous) and maturity is one year. We further assume that the underlying $S$ follows a **Geometric Brownian motion**.

Hint: The terminal stock price $S_T$, under the risk-neutral measure, follows a log-normal distribution with PDF

$$f(x) = \frac{1}{x s \sqrt{2 \pi}} \exp\left( -\frac{(\ln x - \mu)^2}{2 s^2} \right) $$

where $\mu = \ln S_0 + (r-\sigma^2 / 2)T$ and variance $s^2 = \sigma^2 T$.


In [10]:
import numpy as np                                              # Import numpy for numerical calculations
from scipy.integrate import quad                                # Import quad for numerical integration
import math                                                     # Import math for mathematical constants and functions

S0 = 220                                                         # Initial stock price
K = 220                                                          # Strike price
T = 1                                                            # Time to maturity (1 year)
r = 0.10                                                         # Risk-free rate
sigma = 0.98                                                     # Volatility

mu = np.log(S0) + (r - 0.5 * sigma**2) * T                       # Calculate mu 
s2 = sigma**2 * T                                           # Calculate s 

def integrand(x):
    return (x - K) * np.exp(-0.5 * ((np.log(x) - mu)**2 / s2)) / (x * np.sqrt(s2) * np.sqrt(2 * math.pi))

upper_bound = K + 10 * s2                                        # Set a high upper bound to capture tail
expected_payoff, _ = quad(integrand, K, upper_bound)            # Use scipy's quad to integrate the payoff function


call_price = np.exp(-r * T) * expected_payoff                   # Discount the expected payoff at the risk-free rate

print(f"Estimated Call Option Price using numerical integration: {call_price:.2f}") # Output the estimated call price


                                                                
N = 100000                                                                      # Number of simulations

np.random.seed(0)                                                               # Set random seed for reproducibility
Z = np.random.standard_normal(N)                                                # Generate N standard normal random variables
ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)             # Calculate stock prices at T


payoffs = np.maximum(ST - K, 0)                                                 # Calculate payoffs from the call option

call_price = np.exp(-r * T) * np.mean(payoffs)                                  # Discount payoffs to present value and take average

print(f"Estimated Call Option Price using Monte Carlo Simulation: {call_price:.2f}")  # Output the estimated call price

Estimated Call Option Price using numerical integration: 0.07
Estimated Call Option Price using Monte Carlo Simulation: 89.12
